# Analyse MS/MS dataset using MS2DeepScore (and MS2Query)

The goal of this notebook is to show how `MS2DeepScore` can be used to analyse a MS/MS dataset.
We will further use results produced by `MS2Query` to better interpret the outcomes.

In [ ]:
# Necessary to install (if not present yet)
!pip install ms2deepscore  # This will automatically install matchms as well!

# Import libraries

In [ ]:
import os
from matplotlib import pyplot as plt

from matchms import calculate_scores
from matchms.importing import load_from_mgf

from ms2deepscore import MS2DeepScore
from ms2deepscore.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Specify data (and model) locations
path_root = "./drive/MyDrive/Colab_Notebooks"
file = os.path.join(path_root, "data_mzmine", "example_iimn_gnps.mgf")
spectra = list(load_from_mgf()"example_iimn_gnps.mgf"))

In [ ]:
# How many spectra did we get?
len(spectra)

In [ ]:
spectra[0].metadata

## Process spectra
We will apply:
- A few default filters to harmonize metadata
- Normalize peak intensities (to max=1.0)
- Remove all peaks outside 10.0 to 1000.0 Da m/z window
- Remove spectra with < 4 remaining peaks

The last two teps are potentially important for tools like `Spec2Vec` or `MS2DeepScore` which make predictions solely based on the MS2 peaks.

In [ ]:
from matchms import SpectrumProcessor

min_number_peaks = 4

# Define filter pipeline
processor = SpectrumProcessor("basic")
processor.add_matchms_filter("normalize_intensities")
processor.add_matchms_filter(("select_by_mz", {"mz_from": 10, "mz_to": 1000}))
processor.add_matchms_filter(("require_minimum_number_of_peaks", {"n_required": min_number_peaks}))

# Apply filter pipeline
spectra_cleaned = processor.process_spectrums(spectra)
len(spectra_cleaned)

In [ ]:
spectra_cleaned[0].metadata

## MS2DeepScore

`MS2DeepScore` is a deep learning approach to predict Tanimoto score between two MS/MS spectra (without having any chemical information about both compounds obviously...).

See also:
- [MS2DeepScore article](https://jcheminf.biomedcentral.com/articles/10.1186/s13321-021-00558-4)
- [MS2DeepScore Python library (GitHub)](https://github.com/matchms/ms2deepscore)

### Load a pre-trained deep learning model
We will use a model which was trained on positive ionmode data from GNPS (roughly 25.000 compounds, data downloaded on 02/01/2023).
The model can be found on zenodeo: https://zenodo.org/record/8274763 (but will also be in the google drive folder!)

In [ ]:
# Load pretrained model
filename_model = "ms2deepscore_positive_10k_1000_1000_1000_500.hdf5"
model = load_model(os.path.join(path_root, "ms2deepscore_model", filename_model))
ms2ds = MS2DeepScore(model)

### Compute MS2DeepScore Tanimoto score predictions
The following step will compute Tanimoto score predictions for any possible pair of spectra in our dataset.

In [ ]:
# Calculate scores and get matchms.Scores object
scores = calculate_scores(spectra_cleaned, spectra_cleaned, ms2ds, is_symmetric=True)

In [ ]:
# Just to get an idea of what we get
scores.scores.to_array()[:5, :5]

### One example of what to do with those scores: Molecular Networking

In [ ]:
from matchms.networking import SimilarityNetwork

# Define settings
ms2ds_network = SimilarityNetwork(
    identifier_key="scans",
    score_cutoff= ,  # higher numbers produce more isolated sub-graphs
    max_links= ,  # lower number makes sparser networks
    link_method="mutual",  # mutual means: link is only added if in top list of both nodes
)

# Compute the graph (takes some time)
ms2ds_network.create_network(scores)

In [ ]:
# Export to graphml
ms2ds_network.export_to_graphml("ms2ds_graph.graphml")

## MS2DeepScore embeddings for mapping

Embeddings are abstract vectors, usually with the goal to represent data in an abstract vector space. MS2DeepScore aims to "compress" input spectra to such embeddings which finally is also used to compute similarities between two spectra.

However, embeddings can also be used for many other purposes. One of them is to start a further dimensionality reduction based on such embeddings, for instance with algorithms such as t-SNE, UMAP, or TMAP. Here we will use `UMAP` to compute 2D positions for each spectrum in our data based on MS2DeepScore embeddings. The goal of this is to identify groups or clusters of related spectra, as well as to get an impression of the general composition of our dataset.

### Umap
For more details on UMAP see [UMAP documentation](https://umap-learn.readthedocs.io/en/latest/index.html).
Feel free to play with the key parameters (`n_neighbors` and `min_dist`) --> https://umap-learn.readthedocs.io/en/latest/parameters.html

In [ ]:
!pip install umap-learn

In [ ]:
# Compute MS2DeepScoe embeddings (takes a few minutes)
ms2ds_vectors = ms2ds.calculate_vectors(spectra_cleaned)

In [ ]:
import umap

reducer = umap.UMAP(random_state=42,  # this or whatever your favorite number is
                    n_neighbors= ,  # key parameters
                    min_dist= ,
                    )
reducer.fit(ms2ds_vectors)

In [ ]:
embedding_umap = reducer.transform(ms2ds_vectors)
embedding_umap.shape

In [ ]:
# Get some labels and let's start with something simple:
masses = [s.get("precursor_mz") for s in spectra_cleaned]
masses = [s if s is not None else 0 for s in masses]

In [ ]:
import pandas as pd
scans = [s.get("scans") for s in spectra]
data_plot = pd.DataFrame({"x": embedding_umap[:, 0],
                          "y": embedding_umap[:, 1],
                          "scan": [s.get("scans") for s in spectra_cleaned],
                          "precursor-mz": masses,
                          })
data_plot.head()

In [ ]:
import plotly.express as px
import numpy as np

# Create scatter plot using Plotly Express
fig = px.scatter(data_plot,
    x="x",
    y="y",
    color="precursor-mz",
    color_continuous_scale="viridis",
    size_max=50,
    opacity=0.5,
    title='UMAP projection MS2DeepScore embeddings',
    hover_data={"x": False,
                "y": False,
                "scan": True,
                "precursor-mz": True},
    width=800,
    height=800,
)

# Display the figure
fig.show()

---
## MS2Query

Finally, let's use results obtained with **MS2Query** to help interpreting our data. MS2Query essentially uses a Random Forest model that aims to finds the chemically most similar compounds in a library based on scores such as MS2DeepScore and Spec2Vec, but also mass differences etc.


In [ ]:
file_ms2query = os.path.join(path_root, "results_ms2query", "ms2query_results_example.csv")
data_ms2query = pd.read_csv(file_ms2query)
data_ms2query.head()

In [ ]:
data_ms2query.ms2query_model_prediction.hist(bins=20, rwidth=0.8)
plt.xlabel("MS2Query score")

## UMAP mapping of spectra (with MS2Query based labels)

In [ ]:
import pandas as pd

data_plot = pd.DataFrame({"x": embedding_umap[:, 0],
                          "y": embedding_umap[:, 1],
                          "feature_id": data_ms2query.feature_id.values,
                          "superclass": data_ms2query.cf_superclass.values,
                          "npc_pathway": data_ms2query.npc_pathway_results.values,
                          "potential_analoque:": data_ms2query.analog_compound_name.values,
                          "ms2query_tanimoto_prediction": data_ms2query.ms2query_model_prediction.values,
                          })
data_plot.head()

In [ ]:
import plotly.express as px
import numpy as np

# Create scatter plot using Plotly Express
fig = px.scatter(data_plot,
    x="x",
    y="y",
    color=# here column from data_plot,
    size_max=20,
    opacity=0.5,
    title='UMAP projection MS2DeepScore embeddings',
    hover_data={"x": False,
                "y": False,
                "feature_id": True,
                "potential_analoque:": True,
                "ms2query_tanimoto_prediction": True},
    width=900,
    height=700,
)

# Display the figure
fig.show()

In [ ]:
# select high ms2query scores --> most reliable hits
mask = (data_ms2query.ms2query_model_prediction.values > 0.85) \
  & (data_ms2query.precursor_mz_difference < 1)

# Create scatter plot using Plotly Express
fig = px.scatter(data_plot[mask],
    x="x",
    y="y",
    color=# here column from data_plot,
    size_max=50,
    opacity=0.5,
    title='UMAP projection MS2DeepScore embeddings',
    hover_data={"x": False,
                "y": False,
                "feature_id": True,
                "potential_analoque:": True,
                "ms2query_tanimoto_prediction": True},
    width=900,
    height=700,
)

# Display the figure
fig.show()